In [2]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dimod
from collections import defaultdict
import networkx as nx
from itertools import combinations
import datetime
import time

In [3]:
def get_model(nodes, edges, seed=0.2, gamma=80):
     
    G = nx.gnm_random_graph(nodes,edges,seed)

    Q = defaultdict(int)

    # Fill in Q matrix
    for u, v in G.edges:
        Q[(u,u)] += 1
        Q[(v,v)] += 1
        Q[(u,v)] += -2

    for i in G.nodes:
        Q[(i,i)] += gamma*(1-len(G.nodes))

    for i, j in combinations(G.nodes, 2):
        Q[(i,j)] += 2*gamma
    
    return G, Q

In [6]:
nodes = 70
edges = round(nodes * (nodes-1)/2 * 0.2)
seed = 2
prob=0.2
annealing_time=10

for i in range(10):
    graph, qubo_matrix = get_model(nodes, edges, seed=seed)

    sampler = EmbeddingComposite(DWaveSampler(solver={'topology__type': 'pegasus'}))
    
    start = time.time()
    task = sampler.sample_qubo(qubo_matrix,annealing_time=annealing_time,num_reads=10)
    end = time.time()
    runtime = end - start
    
    QpuSamplingTime = task.info['timing']['qpu_sampling_time']
    QpuAccessTime = task.info['timing']['qpu_access_time']
    QpuProgrammingTime = task.info['timing']['qpu_programming_time']

    print('{}  {}  {}  {}'.format(runtime,QpuProgrammingTime/1000,QpuSamplingTime/1000,QpuAccessTime/1000))



87.51672768592834  26.066  1.383  27.448
46.415823459625244  26.076  1.359  27.435
39.49354076385498  26.061  1.334  27.395
54.056803703308105  26.063  1.108  27.171
135.6921021938324  26.059  1.257  27.316
35.98463296890259  26.052  1.435  27.487
43.07421326637268  26.062  1.293  27.355
89.68975591659546  26.064  1.098  27.162
72.88030862808228  26.052  1.51  27.562
38.15349006652832  26.057  1.093  27.15


In [12]:
0.07389092445373535  10.942  0.229  11.171
0.12908601760864258  10.914  2.289  13.204

{'timing': {'qpu_sampling_time': 229,
  'qpu_anneal_time_per_sample': 10,
  'qpu_readout_time_per_sample': 198,
  'qpu_access_time': 11171,
  'qpu_access_overhead_time': 1639,
  'qpu_programming_time': 10942,
  'qpu_delay_time_per_sample': 21,
  'total_post_processing_time': 467,
  'post_processing_overhead_time': 467,
  'total_real_time': 11171,
  'run_time_chip': 229,
  'anneal_time_per_run': 10,
  'readout_time_per_run': 198},
 'problem_id': 'cf020838-53fc-4626-9469-45990478f68f'}

In [16]:
task.info

{'timing': {'qpu_sampling_time': 2289,
  'qpu_anneal_time_per_sample': 10,
  'qpu_readout_time_per_sample': 198,
  'qpu_access_time': 13204,
  'qpu_access_overhead_time': 1578,
  'qpu_programming_time': 10914,
  'qpu_delay_time_per_sample': 21,
  'total_post_processing_time': 60,
  'post_processing_overhead_time': 60,
  'total_real_time': 13204,
  'run_time_chip': 2289,
  'anneal_time_per_run': 10,
  'readout_time_per_run': 198},
 'problem_id': '5c9de0d2-1e11-4998-a73f-cfd98d41cbfe'}